## 들어가기 전에
전에는 사진들이 무슨 사진인지에 대해 알고있었기 때문에 각 과일의 평육느일 구할 수 있었다.
하지만 진짜 비지도 학습에서는 사지에 어떤 과일이 들어있는지 알지 못한다.

이런 경우, k-평균 군집 알고리즘이 평균값을 자동으로 찾아준다. 이 평균값이 클러스터의 중심에 위치하기 때문에 클러스터 중심, 또는 센트로이드라고 부른다.

## k-평균 알고리즘
1. 무작위 k개의 클러스터를 중심으로 정함
2. 각 샘플에서 가장 가까운 클러스터 중심을 찾아 해당 클러스터의 샘플로 지정
3. 클러스터에 속한 샘플의 평균값으로 클러스터 중심을 변경
4. 클러스터 중심에 변화가 없을 때까지 2번으로 돌아가 반복



In [ ]:
!wget https://bit.ly/fruits_300 -O fruits_300.npy

In [ ]:
import numpy as np

fruits = np.load('fruits_300.npy')
fruits_2d = fruits.reshape(-1, 100*100)

npy 파일을 읽고 k-평균 모델을 훈련하기 위해 (샘플 개수, 너비, 높이) 형태의 3차원 배열을 (샘플 개수, 너비x높이)크기를 가진 2차원 배열로 변경한다.

사이킷런의 k-평균 알고리즘은 sklearn_cluster 모듈 아래 KMeans 클래스에 구현되어있다. 또 n_clusters는 클러스터의 개수를 지정하는 변수로 한다.

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 3, random_state = 42)
km.fit(fruits_2d)

군집 결과는 KMeans 클래스 객체의 labels_ 속성에 저장된다.

In [ ]:
print(km.labels_)

레이블된 0,1,2의 값은 어떤 의미도 가지고있지 않다.

In [ ]:
print(np.unique(km.labels_, return_counts=True))

이걸로 통해 클러스터 0이 91개 1이 98개 2가 111개를 모았다는 것을 알 수 있다.

In [ ]:
import matplotlib.pyplot as plt
def draw_fruits(arr, ratio = 1):
    n = len(arr) # arr = 샘플의 개수
    # 한 줄에 10개씩 이미지를 그림, 샘플의 개수를 10으로 나누어 전체 행 개수 계산
    rows = int(np.ceil(n/10))
    # 행이 1개이면 열의 개수는 샘플의 개수, 그렇지않으면 10개
    cols = n if rows < 2 else 10
    fig, axs = plt.subplots(rows, cols, figsize = (cols*ratio, rows*ratio), squeeze=False)
    for i in range(rows):
        for j in range(cols):
            if i*10 + j < n: # n개까지만 그린다
                axs[i,j].imshow(arr[i*10 + j], cmap='gray_r')
            axs[i,j].axis('off')
    plt.show()

draw_fruits()함수는 (샘플개수, 너비, 높이) 3차원 배열을 입력으로 받아 가로로 10개씩 이미지를 출력하는 함수

샘플의 개수에 따라 행과 열의 개수를 계산하고 figsize를 지정. ratio에 따라 크기 달라짐.

이 함수를 사용해 레이블이 0인 과일 사진을 그릴 수 있다. km.labels_ == 0과 같이 쓰면 km.labes_ 배열에서 값이 0인 위치는 True, 그외는 False가된다. 넘파이는 불리언 배열을 사용해 원소를 선택할 수 있다. 이는 불리언 인덱싱 이라고 한다.

In [ ]:
draw_fruits(fruits[km.labels_ == 0])

In [ ]:
draw_fruits(fruits[km.labels_ == 1])

In [ ]:
draw_fruits(fruits[km.labels_ == 2])

## 클러스터의 중심
KMeans 클래스가 최종적으로 찾은 클러스터의 중심은 cluster_centers_ 속성에 저장되어 있다. 이 배열은 fruits_2d 샘플의 클러스터 중심이기 때문에 이미지로 출력하기 위해선 100x100 크기의 2차원 배열로 바꿔야한다


In [ ]:
draw_fruits(km.cluster_centers_.reshape(-1, 100, 100), ratio = 3)

KMeans 클래스는 훈련 데이터 샘플에서 클러스터 중심까지 거리로 변환해주는 transform() 메서드를 가지고 있다. 이는 특성값을 변환하는 도구로 사용할 수 있다.

인덱스가 100인 샘플에 transform() 메서드를 적용해 fit() 메서드와 마찬가지로 2차원 배열을 기대한다. fruits_2d[100]처럼 쓰면 (10000,) 크기의 배열이 되므로 에러 발생. 슬라이싱 연산자를 통해 (1,10000) 크기의 배열을 전달. 하나의 100x100 크기의 샘플을 전달하는 것이다.

In [ ]:
print(km.transform(fruits_2d[100:101]))

하나의 샘플을 전달했기 때문에 반환된 배열은 크기가 (1, 클러스터 개수)인 2차원 배열이다. 첫번째 클러스터(레이블0), 두번째 클러스터(레이블1)가 각각 첫번째 원소, 두번째 원소의 값이다. 세번째 클러스터까지의 거리가 3393.~~ 이다. 이 샘플은 레이플2에 속한 것이다.(가장 가깝기 때문에)

KMeans 클래스는 가장 가까운 클러스터 중심을 예측 클래스로 출력하는 predict() 메서드를 제공


In [ ]:
print(km.predict(fruits_2d[100:101]))

레이블 2로 예측되는 것을 볼 수 있다.

In [ ]:
draw_fruits(fruits[100:101])

k-평균 알고리즘은 반복적으로 클러스터 중심을 옮기면서 최적의 클러스터를 찾는 것. 반복한 횟수는 KMeans 클래스의 n_iter_ 속성에 저장된다.

In [ ]:
print(km.n_iter_)

## 최적의 k 찾기

k-평균 알고리즘의 단점은 클러스터 개수를 사전에 지정해야 한다는 것이다.
실전에서는 몇개의 클러스터가 필요한지 알 수 없다. 

하지만 적절한 클러스터 개수를 찾기위한 대표적인 방법인 '엘보우' 방법이 있다.

앞에서 k-평균 알고리즘은 클러스터 중심과 샘플 사이의 거리를 잴 수 있는 transform() 메서드가 있었다. 이 거리의 제곱의 합을 '이너셔'라고 부른다.
이너셔는 클러스터의 속한 샘플이 얼마나 가깝게 모여 있는지를 나타내는 값으로 생각할 수 있다. 일반적으로 클러스터 개수가 늘어나면 클러스터 개개의 크기는 줄어들기 때문에 이너셔도 같이 줄어든다. 엘보우 방법은 클러스터 개수를 늘려가면서 이너셔의 변화를 관찰하여 최적의 클러스터 개수를 찾는 방법이다.
(이너셔는 클러스터의 샘플이 얼마나 가깝게 있는지를 나타내는 값으로 볼 수 있다)

클러스터 개수를 증가시키면서 이너셔를 그래프로 그리면 이너셔가 감소하는 속도가 꺾이는 지점이 생긴다. 이 지점부터 클러스터 개수를 늘려도 클러스터에 밀집된 정도가 크게 변하지 않는다.
이말은 즉 이너셔가 크게 줄어들지 않는다는 것이다. 이 지점이 팔꿈치 모양이라서 '엘보우'방법이라고 부른다.

아까의 과일 데이터 셋을 이용해 이너셔를 계산해본다. KMeans 클래스는 자동으로 이너셔를 계산해 inertia_ 속성으로 제공한다. 다음 코드는 클러스터 개수 k를 2~6으로 바꿔가며 훈련. fit() 메서드 사용 후 inertia_에 속성을 저장하여 리스트에 추가.

In [ ]:
inertia = []
for k in range(2,7):
    km = KMeans(n_clusters = k, random_state= 42)
    km.fit(fruits_2d)
    inertia.append(km.inertia_)
plt.plot(range(2,7), inertia)
plt.show()

그래프에서 꺾이는 부분이 k=3에서 기울기가 조금 바뀐 것을 볼 수 있다.